In [4]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.layers import Input, Dense, Flatten
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

weights_dict = {}
weight_callback = tf.keras.callbacks.LambdaCallback(
    on_epoch_end=lambda epoch, logs: weights_dict.update({epoch: model.get_weights()}))


In [5]:
# Load and preprocess CIFAR-10 data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Normalize pixel values to [0, 1]
y_train, y_test = to_categorical(y_train), to_categorical(y_test)

# Modify ResNet-50 for CIFAR-10
input_layer = Input(shape=(32, 32, 3))
resnet50_base = ResNet50(weights='imagenet', include_top=False, input_tensor=input_layer)
x = Flatten()(resnet50_base.output)
x = Dense(256, activation='relu')(x)
output_layer = Dense(10, activation='softmax')(x)

model = Model(inputs=input_layer, outputs=output_layer)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 32, 32, 3)]          0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 38, 38, 3)            0         ['input_2[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 16, 16, 64)           9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 16, 16, 64)           256       ['conv1_conv[0][0]']          
 on)                                                                                        

In [14]:
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=20, batch_size=128, validation_data=(x_test, y_test), callbacks=[weight_callback])

Epoch 1/20



KeyboardInterrupt



In [8]:
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f"Test accuracy : {test_accuracy}")

313/313 [==============================] - 23s 74ms/step - loss: 1.1809 - accuracy: 0.6993
Test accuracy : 0.6992999911308289


In [12]:
deltas = []
prev_weights = []
for epoch, weights in weights_dict.items():
    if(epoch == 0):
        prev_weights = np.array(weights, dtype=object)
        continue
    deltas.append((np.array(weights, dtype=object) - prev_weights)/prev_weights)
    prev_weights = np.array(weights, dtype=object)

deltas = np.array(deltas)
epochIdx = 0

for epoch_deltas in deltas:
    epoch_delta_sum = 0
    epoch_delta_size = 0
    for layer_deltas in epoch_deltas:
        epoch_delta_sum += (layer_deltas > 0.009).sum()
        epoch_delta_size += np.size(layer_deltas)
    print("Epoch:", epochIdx, "- {:.3f}".format(epoch_delta_sum/epoch_delta_size))
    epochIdx += 1

/tmp/ipykernel_124742/4242131339.py:7: RuntimeWarning: invalid value encountered in divide
  deltas.append((np.array(weights, dtype=object) - prev_weights)/prev_weights)


Epoch: 0 - 0.304
Epoch: 1 - 0.292
Epoch: 2 - 0.290
Epoch: 3 - 0.308
Epoch: 4 - 0.309
Epoch: 5 - 0.282
Epoch: 6 - 0.295
Epoch: 7 - 0.285
Epoch: 8 - 0.297


In [ ]:
# Checkpoint Model at Each Epoch and Selectively Load Weights